# Lab 5 - Exploring Azure

## Welcome

This is it. This is our final lab together and one that is entirely unique in terms of approach and expectations. I'll explain all of that in a moment, but first I wanted to congratulate all of you.

#### Well done.

This course - _this quarter_ - is a difficult one and all of you have put in the time and effort to get through it. For some of you, python might have clicked immediately; for others, it can take time... _it can still be happening_. But, just like with writing, the more you do it, the better you'll become.

As I've stressed repeatedly, rather than memorizing every piece of syntax, make sure you focus on **how** computation works. The core ideas of flow control, iteration, abstraction... functions, classes, and methods. These ideas aren't unique to python and if you can grasp them; then, you're able to work in almost any programming language. **Yes**, there will be hiccups, but if you can _think computationally_ if you can break apart a big problem into the steps that a computer can follow - then you can accomplish amazing things.

#### Which brings us back to this lab.

Unlike in your other labs, this lab is much more like a task you'd face at a job. Yes, the outcome is a bit silly, but the *linking together of tools in the cloud through a programming language* is **exactly** the skills you'll be expected to have. What you're learning here really is a skillset that's at the forefront of how geospatial technologies are moving. We're going to be working directly with REST APIs and virtual machines, intergrating our work flow into the computational power available through cloud services.

But, that opportunity comes with a cost. Unlike in our other labs, things will break, *they might not go as planned*. In the past, a unique error might pop up (as we are all running complicated systems that leverage lots of different libraries), but fundamentally I knew what the solution _should be_ every time. It was 'canned' in a sense; that's not what's happening here. In this case, you're working with real data and live systems. 

It's a risk, but it comes with an awfully big reward.
#### So what does that mean?
Well, it means that grade wise you are all going to do fine. Let's just get that out of the way. It means that we have to trust one another and help one another through this process.

And, with all of that in mind, let's begin.

## Getting set up.

As always, you're going to want to build your environment.
I would recommend you use python 3.7 and that you install jupyter, folium, and geopandas. I used the `-c conda-forge` to install all three at once, but do what works for you.

In addition, you're going to need to install the azure services we're going to be using. 
`pip install azure-cognitiveservices-search-newssearch`

You'll also have access to their maps service and their sentiment analysis (although you could also use one of our other mapping, geocoding, or sentiment analysis services instead - such as folium, geopy, or NLTK).

## Our goal and some guidance

Here's where things get tricky. Unlike in other labs, I don't have a set of questions for you to work through. Instead, I'm going to give you an end goal and then some tips on how you might approach it. I'll also be constantly visiting each group to help you generate ideas, goals, and methods.

For what it's worth, this is how your Cartography seminar will work, so think of this as a bit of a taste of what's to come. **Have fun**. Be bold and be creative. 

### Your task: Create a map of how people are talking about a place
**Wait, what?**

What I want you to do is create a map that visualizes how a place is being talked about in the news, on social media, etc.

**Wait, how?**

Well, take a look at [this quickstart guide for sentiment analysis](https://docs.microsoft.com/en-us/azure/cognitive-services/text-analytics/quickstarts/python). You can find another [tutorial here](https://www.pingshiuanchua.com/blog/post/simple-sentiment-analysis-python). That second one uses a few approaches, but I recommend you stick with azure or NLTK. 

Sentiment analysis gives you a very rough, machine-learning createsd sense of the 'emotion' (positive or negative) in a given set of texts. It's extremely useful, for example, if you are trying to monitor how people are talking about your business(es) online. 

Then take a look at [this guide on conducting news searches](https://docs.microsoft.com/en-us/azure/cognitive-services/bing-news-search/news-sdk-python-quickstart). (note: you've already set up your environment as they do at the beginning). 

You can also read about azure maps' geocoding service [here](https://docs.microsoft.com/en-us/rest/api/maps/search). You have access to this service (see the passkeys below).

But, remember, you have other sources of information as well! You could - for example - monitor twitter for tweets about a place (talk to me if you'd like to do this); you could use the geopy geocoder (you've seen examples of this in my code, you can read more about it [here](https://geopy.readthedocs.io/en/stable/); you could - being careful with your credits - use the Arc GIS API geocoder or other services.

**At this point, you have a lot of tools open to you**. Talk with your group, talk with me, come up with a plan.

#### Your final output must have:
    1. At least 250 entries (so 250 news stories, tweets, instagram posts, whatever).
    2. Some indication of the 'sentiment' of the entry (it can be as simple as 'blue for positive, red for negative' push pins; obviously, more interesting results are better)
    

### A sample workflow

If you're stuck, you can follow along with me as I biuld something during the lab.

Here's the workflow I'll follow:
1. Conduct a news search about an area (probably Tacoma, but if that doesn't get enough results somewhere else).
2. Run the results of that search through the sentiment analysis
3. Map it
        a. First, just simply putting points on a map with the color related to the sentiment.
        b. Then, getting fancier. I'll try to create a heat map and then even normalize that heat map based on sentiment or population or something. Who knows where we'll end up, it's exciting to poke at code.
        
        
You can find the keys for the existing services in a password protected file [here](https://github.com/UWTMGIS/GIS501_w19_files/blob/master/w19_azurekeys.pdf.zip). I will provide the password in class.
    

# Let's go!
## Get fancy, get creative, have fun.

In [1]:
#https://www.youtube.com/watch?v=3bxGHYOeNZA 
from geopy import geocoders
from geopy.geocoders import Nominatim
from textblob import TextBlob
import tweepy
import pandas as pd 
import csv
import time 
import re

    
CK = 'KEY'
CS = 'KEY'
AK = 'KEY'
AS = 'KEY'

auth = tweepy.OAuthHandler(CK, CS)
auth.set_access_token(AK, AS)

#sets limit to true, waits for limit to continue
api = tweepy.API(auth, wait_on_rate_limit=True, wait_on_rate_limit_notify=True)


#Creating a function to remove URLs in tweets 
def remove_url(txt):
    """Replace URLs found in a text string with nothing 
    (i.e. it will remove the URL from the string).

    Parameters
    ----------
    txt : string
        A text string that you want to parse and remove urls.

    Returns
    -------
    The same txt string with url's removed.
    """

    return " ".join(re.sub("([^0-9A-Za-z \t])|(\w+:\/\/\S+)", "", txt).split())

time.sleep(1)
i = 3
while i!=0: 
    print("LoAdIng ~",i,"~")
    time.sleep(1)
    i=i-1 

    
public_tweets= tweepy.Cursor (api.search,
                              q='coronavirus -filter:retweets', 
                              lang="en").items(400)
#It doesn't print the exact numbers. It usually cut it by half of what we put in. 

def geo(location):
    g = geocoders.Nominatim(user_agent='virus', timeout=None)
    try:
        loc = g.geocode(location)
        return loc.latitude, loc.longitude
    except:
        return 0,0


#Create a list to append things to 
userList = []
textList = []
polarityList = []
subjectivityList = []
sentimentList = [] 
latList = []
lonList = []



for tweet in public_tweets:
    if tweet.user.location is not None:
        Lat,Lon = geo(tweet.user.location)
        if Lat!=0: 
            latList.append(Lat)
            lonList.append(Lon)
            user = tweet.user.screen_name
            userList.append(user)
            txt = remove_url(tweet.text)
            textList.append(txt)
            analysis = TextBlob(txt)  
            polarity = analysis.sentiment.polarity 
            subjectivity = analysis.sentiment.subjectivity 
            polarityList.append(polarity)
            subjectivityList.append(subjectivity)
            if(polarity == 0):
                sentimentList.append('Gray')
            elif(polarity > 0): 
                sentimentList.append('Green')
            elif(polarity < 0):
                sentimentList.append('Red')
            print(txt)
            print ('Polarity: ', polarity)


dataFrame = pd.DataFrame({
    'User':userList, 
    'Tweet':textList, 
    'Polarity': polarityList,
    'Sentiment': sentimentList, 
    'Lat': latList, 
    'Lon': lonList})

dataFrame.head(15)

LoAdIng ~ 3 ~
LoAdIng ~ 2 ~
LoAdIng ~ 1 ~
9210 NIOSH Approved N95 Fold Flat Mask Respirator 1 Masks Individually packedCOVID19
Polarity:  -0.0125
Can anyone confirm this coronavirus COVID19
Polarity:  0.0
WTF I get that this coronavirus is a huge deal and should be taken seriously but it doesnt give you any right to
Polarity:  -0.036904761904761885
New York AG Warns Televangelist Jim Bakker To Stop Selling Fake Coronavirus Cure According to this fraudster his
Polarity:  -0.18181818181818182
Patients of sick doctor in Melbourne traced by coronavirus authorities as it happened
Polarity:  -0.7142857142857143
NEW CORONAVIRUS CONSPIRACYWhy does the Apple emoji for a germ looks IDENTICAL to what the coronavirus looks like
Polarity:  0.13636363636363635
SenSanders Okay seriously Im a nurse and Im not worried about the Coronavirus 1 if you know how to wash your
Polarity:  0.08333333333333334
Fake news She and Williams are supposedly educated and perpetuated a hoax fake news telling people this

Which essential oil is best for coronavirus Want to get stocked up before its all gone
Polarity:  0.5
CoronavirusEveryone freaking outMe lets drive to Chicago for the week for school brings 4yearold daughte
Polarity:  0.0
PhilipRucker I promise you President Trump has no higher priority than the health and safety of the American peop
Polarity:  -0.0625
VP does it feel weird that Trump is trying to pin Coronavirus on you after all the embarrassment and shame youve endured as his lapdog
Polarity:  -0.5
Dr Ben Carson Brilliantly Turns Tables on Dems Attacking Mike Pences Coronavirus Response
Polarity:  0.9
Just bought tickets to the OKC game on Weds night CoronaVirus stay away yoTakeNote WashYourHands NoFaceTouching
Polarity:  -0.4
NicoleLeighAnn6 ALISHAMARIE the flu is not worse the death rate for the coronavirus is higher and continuing to
Polarity:  0.225
thehill Maybe he heard novel coronavirus and thought that meant it was a work of fiction
Polarity:  0.0
Ask2020 FLU in the US 7Feb20

USC will temporarily move classes online from March 1113 to test technical capabilities No confirmed cases of th
Polarity:  -0.1
Me I cant make it tonight Them I get it worried about the coronavirus Me right riiight
Polarity:  0.2857142857142857
YahooSports Chargers organization moved to LA to cut down on fans Success Coronavirus
Polarity:  0.0722222222222222
After listening to Erin Burnett cough during her CNN segment tonight on the coronavirus Im starting to wonder if
Polarity:  0.0
Doctors using artificial intelligence to track coronavirus outbreak datascience deeplearning
Polarity:  -0.6
Successful remote teams need more than tools Ive been coaching companies on effective remote team strategies for
Polarity:  0.32999999999999996
i went to a local chinese place to pick up food amp some ignorant fuck yelled no coronavirus please into the door
Polarity:  -0.13333333333333333
i swear if they cancel fiesta because of the coronavirus im gonna be soo mad
Polarity:  -0.625
For our new stor

TitusNation Trump cannot successfully lie or cover up here In other situations he demands others to be loyal to
Polarity:  0.3194444444444444
Currently at Target in Irvine and theyre sold out of many basic needs like toilet paper common cold medicines an
Polarity:  -0.07222222222222222
Heres whyLets say youre a waiter You make minimum wage plus tips averaging oh 20hour You call in sick
Polarity:  -0.7142857142857143
Maybe now theyll pay attention A Scientific Adviser On Contagion Said Peopl
Polarity:  0.0
How does the recent spread of COVID19 outside of China compare to the outbreak within China a few weeks ago Datas
Polarity:  -0.06666666666666667
um Erinburnett is coughing up a lung rn on CNN as she covers CoronaVirus Everyone ignores it including
Polarity:  0.0
COVID19 is an emerging infectious disease which means information is changing from minute to minuteThe NJ Cor
Polarity:  0.0
Most schools wont close unless told to do so bc KCPubHealth youre the experts Kids falling behind if

,User,Tweet,Polarity,Sentiment,Lat,Lon
0,QuadeersultanM,9210 NIOSH Approved N95 Fold Flat Mask Respira...,-0.012500,Red,17.388786,78.461065
1,thomasheimann,Can anyone confirm this coronavirus COVID19,0.000000,Gray,39.783730,-100.445882
2,atypicalidiot,WTF I get that this coronavirus is a huge deal...,-0.036905,Red,55.001251,-97.001038
3,forwardarc,New York AG Warns Televangelist Jim Bakker To ...,-0.181818,Red,36.701463,-118.755997
4,davidh7426,Patients of sick doctor in Melbourne traced by...,-0.714286,Red,53.407901,-2.160243
5,DontJAY_MyKNEE,NEW CORONAVIRUS CONSPIRACYWhy does the Apple e...,0.136364,Green,33.749099,-84.390185
6,caitstlyrn,SenSanders Okay seriously Im a nurse and Im no...,0.083333,Green,43.979280,-120.737257
7,MarijaneGreen,Fake news She and Williams are supposedly educ...,-0.466667,Red,39.783730,-100.445882
8,RobertDEdwards,Florida has reported its first two coronavirus...,0.250000,Green,38.652954,-90.241117
9,sonnerly,something very interesting about how canada ha...,0.575000,Green,51.000678,0.416139


In [2]:
dataFrame.to_csv('corona3.csv')

In [8]:
# https://stackoverflow.com/questions/56876620/unsure-how-to-use-colormap-with-folium-marker-plot
import folium
from folium import plugins
import branca
import branca.colormap as cm
import pandas as pd 
import geopandas


tweets = pd.read_csv('corona3.csv')
panda = pd.DataFrame(tweets, columns=['User','Tweet','Polarity','Sentiment','Lat','Lon'])


m = folium.Map(
                location=[0,0],
                zoom_start=2,
                tiles='cartodb positron')

#Creating the color scale for polarity 
colormap = cm.LinearColormap(colors=['red','green'], index=[-1,1],vmin=-1,vmax=1)


for index, row in tweets.iterrows():
    folium.CircleMarker(
        location=[row['Lat'], row['Lon']],
        radius=6,
        weight=4,
        fill=True,
        popup=folium.Popup(row['User'] + '<br>' + row['Tweet'],
                          max_width=450, max_height=400), 
        color=colormap(row['Polarity']) 
    ).add_to(m)
                  
m.add_child(colormap)

m